# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for py4j==0.10.9.7 from https://files.pythonhosted.org/packages/10/30/a58b32568f1623aaad7db22aa9eafc4c6c194b429ff35bdc55ca2726da47/py4j-0.10.9.7-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 4.9 MB/s eta 0:00:00ta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840633 sha256=cd47b2ac374164f45bd6cf4a2a7eafb736669a2830c1cc4021bdfc9e57281093
  Stored in directory: /Users/mac/Library/Caches/pip/wheels/97/f5/c0/947e2c0942b361ffe58651f36bd7f13772675b3863fd63d1b1
Successfully built pyspark


In [83]:
from pyspark.sql import SparkSession, functions as F, Window
from pyspark.sql.functions import split, col, lower, row_number, collect_list, struct, sort_array, expr, concat_ws
import json

In [5]:
# Initialize Spark session in local mode
spark = SparkSession.builder \
    .appName("ClickstreamRouteAnalysis") \
    .master("local[2]") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

24/10/28 02:50:51 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.Con

In [87]:
df = spark.read.option("delimiter", "\t").csv("clickstream.csv", header=True)


In [88]:
df.show(20)

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584160|
|    562|       507|        page|    rabota|1695584166|
|    562|       507|       event|    rabota|1695584174|
|    562|       507|       event|    rabota|1695584181|
|    562|       507|       event|    rabota|1695584189|
|    562|       507|        page|      main|1695584194|
|    562|       507|       event|      main|1695584204|
|    562|       507|       event|      main|1695

## Spark DF

In [90]:
# Identify error events by checking if the event_type contains 'error' (case insensitive), flagging them as 1 in the 'error_flag' column
df = df.withColumn("error_flag", F.when(F.lower(F.col("event_type")).contains("error"), 1).otherwise(0))

# Define a window that partitions by user and session, ordering by timestamp to calculate a cumulative sum of error flags within each session
window_spec = Window.partitionBy("user_id", "session_id").orderBy("timestamp").rowsBetween(Window.unboundedPreceding, Window.currentRow)
df = df.withColumn("cumulative_error", F.sum("error_flag").over(window_spec))

# Filter out rows after the first error in each session, keeping only those where no cumulative error has occurred, then drop error columns
filtered_df = df.filter(F.col("cumulative_error") == 0).drop("error_flag", "cumulative_error")


In [91]:
# Set up a session window to analyze consecutive events and remove duplicates within each session based on the event page
window_session = Window.partitionBy("user_id", "session_id").orderBy("timestamp")
filtered_df = (
    filtered_df.withColumn("prev_page", F.lag("event_page").over(window_session))  # Check previous event page
               .filter((F.col("event_page") != F.col("prev_page")) | F.col("prev_page").isNull())  # Keep if page is different from previous or if it's the first event
)

In [92]:
# Aggregate the event pages to form a route, then count and rank the most frequent routes, limiting to the top 30 routes
routes_df = (
    filtered_df.groupBy("user_id", "session_id")
               .agg(F.collect_list("event_page").alias("pages"))  # Collect event pages as a list
               .withColumn("route", F.expr("array_join(pages, '-')"))  # Join the list to form a single route
               .groupBy("route")
               .count()
               .orderBy(F.desc("count"))
               .limit(30)
)

In [93]:
top_10_routes_df = routes_df.limit(10)

top_10_routes_dict = {row["route"]: row["count"] for row in top_10_routes_df.collect()}

with open("result.json", "w") as file:
    json.dump(top_10_routes_dict, file, indent=4)

In [94]:
!curl -F file=@result.json 51.250.123.136:80/MDS-LSML1/mlagosha/w6/2

0.8999999999999999
Correct main answer!
Correct main-archive answer!
Correct main-rabota answer!
Correct main-internet answer!
Correct main-bonus answer!
Correct main-news answer!
Correct main-tariffs answer!
Correct main-online answer!
Wrong main-vklad answer!
Correct main-rabota-archive answer!


## Spark RDD 

In [95]:
# Convert DataFrame to RDD for further processing
data_rdd = df.rdd.map(tuple)  # Convert DataFrame rows to tuples

# Identify sessions containing error events and extract the timestamp of each error
session_errors = data_rdd.filter(lambda x: "error" in x[2].lower()).map(lambda x: ((x[0], x[1]), x[4]))

# Get the earliest error timestamp within each session
first_error_ts = session_errors.reduceByKey(lambda a, b: min(a, b))

In [96]:
# Filter events that occur before the first error in each session
data_rdd = data_rdd.map(lambda x: ((x[0], x[1]), x)) \
                   .leftOuterJoin(first_error_ts) \
                   .filter(lambda x: x[1][1] is None or x[1][0][4] < x[1][1]) \
                   .map(lambda x: x[1][0])

In [97]:
# Sort events by user, session, and timestamp to maintain sequence
sorted_rdd = data_rdd.sortBy(lambda x: (x[0], x[1], x[4]))

def deduplicate_consecutive_pages(records):
    previous_page = None
    for record in records:
        if record[3] != previous_page:  
            yield record[3]
        previous_page = record[3]

In [98]:
# Group by user_id and session_id, deduplicate consecutive pages, and filter sessions with at least one page
session_routes = sorted_rdd.map(lambda x: ((x[0], x[1]), x)).groupByKey() \
                           .mapValues(deduplicate_consecutive_pages) \
                           .mapValues(lambda pages: list(pages)) \
                           .filter(lambda x: len(x[1]) >= 1)


route_counts = session_routes.map(lambda x: ("-".join(x[1]), 1)).reduceByKey(lambda a, b: a + b)

top_10_routes = route_counts.takeOrdered(10, key=lambda x: -x[1])
for route, count in top_10_routes:
    print(f"{route}\t{count}")

main	8184
main-archive	1113
main-rabota	1047
main-internet	897
main-bonus	870
main-news	769
main-tariffs	677
main-online	587
main-vklad	518
main-rabota-archive	170


##  Spark SQL

In [100]:
# Mark error events by adding a flag column, making it 1 where event_type contains 'error'
df = df.withColumn("error_flag", F.when(F.lower(F.col("event_type")).like("%error%"), 1).otherwise(0))

In [102]:
df

DataFrame[user_id: string, session_id: string, event_type: string, event_page: string, timestamp: string, error_flag: int, cumulative_error: bigint, error_accumulated: bigint]

In [103]:
error_window = Window.partitionBy("user_id", "session_id").orderBy("timestamp").rowsBetween(Window.unboundedPreceding, Window.currentRow)
df = df.withColumn("error_accumulated", F.sum("error_flag").over(error_window))


In [104]:
filtered_df = df.filter(F.col("error_accumulated") == 0).drop("error_flag", "error_accumulated")


In [105]:
filtered_df

DataFrame[user_id: string, session_id: string, event_type: string, event_page: string, timestamp: string, cumulative_error: bigint]

In [106]:
session_window = Window.partitionBy("user_id", "session_id").orderBy("timestamp")
filtered_df = (
    filtered_df.withColumn("previous_page", F.lag("event_page").over(session_window))
               .filter((F.col("event_page") != F.col("previous_page")) | F.col("previous_page").isNull())
)

In [107]:
filtered_df.createOrReplaceTempView("filtered_clickstream_data")

In [108]:
routes_df = spark.sql("""
    WITH user_session_routes AS (
        SELECT 
            user_id, 
            session_id, 
            COLLECT_LIST(event_page) AS page_sequence
        FROM filtered_clickstream_data
        GROUP BY user_id, session_id
    ),
    route_frequency AS (
        SELECT 
            ARRAY_JOIN(page_sequence, '-') AS route, 
            COUNT(1) AS route_count
        FROM user_session_routes
        GROUP BY route
    )
    SELECT route, route_count
    FROM route_frequency
    ORDER BY route_count DESC
    LIMIT 10
""")

In [109]:
routes_df.show(truncate=False)

+-------------------+-----------+
|route              |route_count|
+-------------------+-----------+
|main               |8185       |
|main-archive       |1113       |
|main-rabota        |1047       |
|main-internet      |897        |
|main-bonus         |870        |
|main-news          |769        |
|main-tariffs       |677        |
|main-online        |587        |
|main-vklad         |517        |
|main-rabota-archive|170        |
+-------------------+-----------+



In [110]:
# Stop the Spark session
spark.stop()